<a href="https://colab.research.google.com/github/Jahid1990/News-BOT/blob/main/Chatbot_Query_Intent_Recognition_for_Bengali%2C_Banglish_and_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
# !pip install bnlp_toolkit
import numpy as np # linear algebra
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
pd.set_option("display.max_rows", 50000, "display.max_columns", 500)

# Data Load and visualization

In [24]:
data=pd.read_csv("/content/drive/MyDrive/to_english.csv")
data=data.iloc[:,2:]
data.columns=["Intent","Text"]
data.head()

,Intent,Text
0,checkavailablebalanceand validity,How much money is in my account?
1,checkavailablebalanceand validity,How do I view the account balance?
2,checkavailablebalanceand validity,How much money do I have in balance?
3,checkavailablebalanceand validity,How much money do I have?
4,checkavailablebalanceand validity,Account is the balance ball


# Frequency Distribution of pre defined Query Intents

In [35]:
data.Intent.value_counts()/data.shape[0]*100

CheckSocialDataofferinfo                       30.631240
Checkdatapackinfo                              28.270999
checkemergencybalance&creditlimit               4.729509
checkfnf&superfnfstatus                         4.602020
checkcurrentdatapackagestatus                   4.146218
checkdatabalanceand validity                    2.097366
checktalktimeandvalidity                        1.774694
checklastrechargeamount&date                    1.659615
checkavailablebalanceand validity               1.473459
checksmsbalanceandvalidity                      1.454279
checkpostpaidcurrent&totaldue                   1.392227
checkstarstatusandvalidity                      1.391098
simpinandpuknumber                              1.278276
checkbillingaddress                             1.262481
Checkpostpaidbillcycle                          1.054888
checkcurrentpackagestatus                       0.955604
Checkpostpaidtotal&availablecreditlimit         0.863090
Checkpostaiddueamountinlastinvo

# Text Processsing

# Learning
1. translation would be helpful for larger text documents
2. charac level and world level giving similar results
3. original/mixed texts working better than translated ones

# Train test split and text processing

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(data.Text,data.Intent, test_size = 0.40, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(53181,) (53181,)
(35454,) (35454,)


In [26]:
max_fatures =2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data.Text)
#tokenizer.index_word=dict(pd.Series(tokenizer.index_word.values())[~pd.Series(tokenizer.index_word.values()).isin(stop_words)])
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,padding='post')

In [27]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,padding='post',maxlen=23)

In [28]:
#Y = pd.get_dummies(data['Intent'],prefix_sep='a')
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
Y_train= pd.get_dummies(Y_train,prefix_sep='a')
Y_test= pd.get_dummies(Y_test,prefix_sep='a')
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(53181, 23) (53181, 33)
(35454, 23) (35454, 33)


# Model Architecture

In [36]:
embed_dim = 196
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(tf.keras.layers.Flatten())
model.add(Dense(512,input_shape=(23,),activation='relu'))
model.add(Dense(Y_train.shape[1],activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 23, 196)           392000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 23, 196)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 23, 200)           317600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 23, 200)           320800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               320800    
_________________________________________________________________
flatten_2 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)              

In [37]:
import tensorflow as tf
learn_rate = 0.001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
#checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/gp_chatbot.h5', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [38]:
batch_size = 512
learn_rate = 0.0001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size,validation_split=.2, verbose = 1)

Epoch 1/50
84/84 [==============================] - 10s 79ms/step - loss: 2.3672 - accuracy: 0.3254 - val_loss: 1.3755 - val_accuracy: 0.6310
Epoch 2/50
84/84 [==============================] - 4s 44ms/step - loss: 1.1089 - accuracy: 0.6689 - val_loss: 0.5651 - val_accuracy: 0.8106
Epoch 3/50
84/84 [==============================] - 4s 44ms/step - loss: 0.5699 - accuracy: 0.8015 - val_loss: 0.4400 - val_accuracy: 0.8475
Epoch 4/50
84/84 [==============================] - 4s 46ms/step - loss: 0.4119 - accuracy: 0.8599 - val_loss: 0.2931 - val_accuracy: 0.9055
Epoch 5/50
84/84 [==============================] - 4s 44ms/step - loss: 0.2982 - accuracy: 0.9006 - val_loss: 0.2913 - val_accuracy: 0.9008
Epoch 6/50
84/84 [==============================] - 4s 44ms/step - loss: 0.2724 - accuracy: 0.9089 - val_loss: 0.2406 - val_accuracy: 0.9199
Epoch 7/50
84/84 [==============================] - 4s 44ms/step - loss: 0.2462 - accuracy: 0.9189 - val_loss: 0.2093 - val_accuracy: 0.9306
Epoch 8/50
8

#Evaluation

In [39]:
from sklearn.metrics import classification_report
import tensorflow as tf
#model=tf.keras.models.load_model('/content/drive/MyDrive/gp_chatbot.h5')
pred=model.predict(X_test)                                                                                                                                                                          
pred=np.argmax(pred,axis=1)
act=np.argmax(Y_test.to_numpy(),axis=1)
print(classification_report(pred, act))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       245
           1       1.00      0.93      0.96     11713
           2       0.94      0.98      0.96       242
           3       0.95      1.00      0.97       242
           4       0.98      0.95      0.96       244
           5       0.94      0.99      0.96       215
           6       0.93      0.99      0.96      9493
           7       0.93      0.99      0.96       257
           8       0.94      0.96      0.95       239
           9       0.93      0.91      0.92       263
          10       0.89      1.00      0.94       223
          11       0.98      0.91      0.94       387
          12       0.94      0.95      0.94       239
          13       0.94      0.81      0.87       308
          14       0.98      0.99      0.98       274
          15       0.85      0.89      0.87       228
          16       0.97      0.89      0.93       260
          17       0.90    

In [41]:
# data["PRED"]=Y_test.columns[pd.DataFrame(pred).idxmax(axis=1)]
# data["PRED_ACC"]=np.where(Y_test.Intent==data.PRED,1,0)
# data["PRED_ACC"].value_counts()/data.shape[0]*100
# data

In [ ]:
data.PRED_ACC.value_counts()/data.shape[0]*100

0    99.299374
1     0.700626
Name: PRED_ACC, dtype: float64

In [42]:
test=['Taka kata acha acaunte',
'Balns kivabe deke',
'bortomane amar SIM er balance kata',
'koto ache sir balance e',
'ami r koto khoroch korte parbo',
'jana jabe ki koto ache',
'কল যাচ্ছে না কেন?',
'দেখবো কিভাবে কত আছে একাউন্টে?',
'একটু আমার বেলেন্স চেক করে বলেন কত আছে?',
'আমার এই নাম্বারে কত আছে দেখে বলেন তো']

In [45]:
!pip install google_trans_new
from google_trans_new import google_translator  
translator = google_translator()  
twt='amer koto ase'
#twt=translator.translate(twt,lang_tgt='en')
twt=[twt]

#vectorizing
twt = tokenizer.texts_to_sequences(test)
# #padding
twt = pad_sequences(twt,padding='post',maxlen=23,value=0.0,dtype='int32')
sentiment = model.predict(twt)
print(Y_test.columns[pd.DataFrame(sentiment).idxmax(axis=1)][0]+":"+" "+str(np.max(sentiment)))

checkavailablebalanceand validity: 0.9995435


In [ ]:
len(tokenizer.word_index)

3963

In [ ]:
twt = 'internet kivabe kinte hoy'
twt=translator.translate(twt,lang_tgt='en')
twt

'How to buy internet '

In [ ]:
data

,Intent,Text,PRED
0,checkavailablebalanceand validity,How much money is in my account?,checkavailablebalanceand validity
1,checkavailablebalanceand validity,How do I view the account balance?,checkavailablebalanceand validity
2,checkavailablebalanceand validity,How much money do I have in balance?,checkavailablebalanceand validity
3,checkavailablebalanceand validity,How much money do I have?,checkavailablebalanceand validity
4,checkavailablebalanceand validity,Account is the balance ball,checkavailablebalanceand validity
...,...,...,...
88630,MicroVoiceOfferInfo,What special minit offer there in my number,MicroVoiceOfferInfo
88631,MicroVoiceOfferInfo,Any minute offer are in my number,MicroVoiceOfferInfo
88632,MicroVoiceOfferInfo,Inform me about my minit offer in this number,MicroVoiceOfferInfo
88633,MicroVoiceOfferInfo,Which menit offer are available in my number,MicroVoiceOfferInfo


In [ ]:
# text=" 10 ".join(data.iloc[:,0])
# #!pip install google_trans_new
# l=4500
# from google_trans_new import google_translator  
# translator = google_translator() 
# t=[]
# for i in range(20):
#   t.append(translator.translate(text[(i*l):(i+1)*l],lang_tgt='en'))

In [ ]:
!pip install punkt

ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [ ]:
translator.translate('আমি বাংলায় গান গাই',lang_tgt='en')

'I sing in Bengali '

# Convert all text to English

In [ ]:
#!pip install google_trans_new

from google_trans_new import google_translator  
translator = google_translator()  
trans=[]
l=0
for i in data.iloc[:,0]:
  trans.append(translator.translate(i,lang_tgt='en')) 
  l=l+1 
  print(l)

In [ ]:
pd.concat([data,pd.DataFrame(trans)],axis=1).to_excel("/content/drive/MyDrive/to_english.xlsx")

In [ ]:
pd.DataFrame(trans).to_excel("/content/drive/MyDrive/to_english.xlsx")

# Tokenizer

In [ ]:
from bnlp import BasicTokenizer
basic_t = BasicTokenizer()
raw_text = "আমি বাংলায় গান গাই।"
tokens = basic_t.tokenize(raw_text)
print(tokens)

# output: ["আমি", "বাংলায়", "গান", "গাই", "।"]

punkt not found. downloading...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['আমি', 'বাংলায়', 'গান', 'গাই', '।']


In [ ]:
from bnlp import NLTKTokenizer

bnltk = NLTKTokenizer()

text = "আমি ভাত খাই। সে বাজারে যায়। তিনি কি সত্যিই ভালো মানুষ?"

word_tokens = bnltk.word_tokenize(text)
sentence_tokens = bnltk.sentence_tokenize(text)
print(word_tokens)
print(sentence_tokens)

['আমি', 'ভাত', 'খাই', '।', 'সে', 'বাজারে', 'যায়', '।', 'তিনি', 'কি', 'সত্যিই', 'ভালো', 'মানুষ', '?']
['আমি ভাত খাই।', 'সে বাজারে যায়।', 'তিনি কি সত্যিই ভালো মানুষ?']


# STOP words

In [ ]:
from bnlp.corpus import stopwords,punctuations
from bnlp.corpus.util import remove_stopwords

# remove stop words
stopwords = stopwords()
raw_text = 'আমি ভাত খাই।'
result = remove_stopwords(raw_text, stopwords)
print(result)
# remove punctuations
final_result = remove_stopwords(raw_text, punctuations)
print(final_result)

['ভাত', 'খাই', '।']
['আমি', 'ভাত', 'খাই']


In [ ]:
from bnlp import BengaliGlove

bng = BengaliGlove()
glove_path = "/content/drive/MyDrive/bn_glove.39M.300d.txt"
word = "গ্রাম"
# res = bng.closest_word(glove_path, word)
# print(res)
vec = bng.word2vec(glove_path, word)


300

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 9.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=b52e9a2a28d218b9dbbf56718e7f48836b2ba16028e14ffdf22b3b1d097533d9
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
>>> from langdetect import detect
>>> detect("এই সেতু শুধু যোগাযোগব্যবস্থা সহজ করবে না")

'bn'